In [83]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [84]:
# Importamos datasets desde la URL de GitHub para evitar la descarga local de los archivos CSV.
url_prices = "https://raw.githubusercontent.com/nicolinolochex/DataScience/main/data/prices-split-adjusted.csv"
url_securities = "https://raw.githubusercontent.com/nicolinolochex/DataScience/main/data/securities.csv"
url_fundamentals = "https://raw.githubusercontent.com/nicolinolochex/DataScience/main/data/fundamentals.csv"

# Cargamos cada dataset desde la URL RAW
df_prices = pd.read_csv(url_prices)
df_securities = pd.read_csv(url_securities)
df_fundamentals = pd.read_csv(url_fundamentals)

# Verificamos algunas filas de cada dataframe
print("Prices Split Adjusted:\n", df_prices.head(), "\n")
print("Securities:\n", df_securities.head(), "\n")
print("Fundamentals:\n", df_fundamentals.head(), "\n")


Prices Split Adjusted:
          date symbol        open       close         low        high  \
0  2016-01-05   WLTW  123.430000  125.839996  122.309998  126.250000   
1  2016-01-06   WLTW  125.239998  119.980003  119.940002  125.540001   
2  2016-01-07   WLTW  116.379997  114.949997  114.930000  119.739998   
3  2016-01-08   WLTW  115.480003  116.620003  113.500000  117.440002   
4  2016-01-11   WLTW  117.010002  114.970001  114.089996  117.330002   

      volume  
0  2163600.0  
1  2386400.0  
2  2489500.0  
3  2006300.0  
4  1408600.0   

Securities:
   Ticker symbol             Security SEC filings             GICS Sector  \
0           MMM           3M Company     reports             Industrials   
1           ABT  Abbott Laboratories     reports             Health Care   
2          ABBV               AbbVie     reports             Health Care   
3           ACN        Accenture plc     reports  Information Technology   
4          ATVI  Activision Blizzard     reports  Informat

In [85]:
"""
Este bloque de código realiza una inspección inicial de cada DataFrame (df_prices, df_securities, df_fundamentals).
1. Muestra el nombre del DataFrame para distinguir la salida.
2. Imprime su forma (Shape: número de filas y columnas).
3. Lista las columnas que contiene
"""
dfs = [
    ("df_prices", df_prices),
    ("df_securities", df_securities),
    ("df_fundamentals", df_fundamentals)
]

for name, df in dfs:
    print(f"--- {name} ---")
    print("Shape:", df.shape)
    print("Columns:", df.columns)


--- df_prices ---
Shape: (851264, 7)
Columns: Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')
--- df_securities ---
Shape: (505, 8)
Columns: Index(['Ticker symbol', 'Security', 'SEC filings', 'GICS Sector',
       'GICS Sub Industry', 'Address of Headquarters', 'Date first added',
       'CIK'],
      dtype='object')
--- df_fundamentals ---
Shape: (1781, 79)
Columns: Index(['Unnamed: 0', 'Ticker Symbol', 'Period Ending', 'Accounts Payable',
       'Accounts Receivable', 'Add'l income/expense items', 'After Tax ROE',
       'Capital Expenditures', 'Capital Surplus', 'Cash Ratio',
       'Cash and Cash Equivalents', 'Changes in Inventories', 'Common Stocks',
       'Cost of Revenue', 'Current Ratio', 'Deferred Asset Charges',
       'Deferred Liability Charges', 'Depreciation',
       'Earnings Before Interest and Tax', 'Earnings Before Tax',
       'Effect of Exchange Rate',
       'Equity Earnings/Loss Unconsolidated Subsidiary', 'Fixed Assets',
    

In [86]:
"""
Este bloque de código realiza una inspección básica de cada DataFrame (df_prices, df_securities, df_fundamentals):
1. Muestra la estructura (df.info()), revelando tipos de datos y conteos de valores no nulos.
2. Imprime la cantidad de valores nulos por cada columna (df.isna().sum()).
Al finalizar, podemos identificar cuáles columnas requieren tratamiento de nulos o ajustes de tipo.
"""

dfs = [
    ("df_prices", df_prices),
    ("df_securities", df_securities),
    ("df_fundamentals", df_fundamentals)
]

for name, df in dfs:
    print(f"--- {name} ---\n")
    df.info()  # Muestra tipos de datos y non-null counts en pantalla
    print("\nValores nulos por columna:")
    print(df.isna().sum())
    print("\n" + "="*80 + "\n")

# Conclusión:
# - df_prices no presenta valores nulos y está bien tipificado, salvo la verificación de volumen = 0.(Se analizo mas adelante, no interfiere en el analisis)
# - df_securities solo tiene nulos en 'Date first added'; se decidirá cómo tratarlos según la relevancia. (Se procedio a eliminar esta columna)
# - df_fundamentals sí presenta nulos en varias columnas financieras (p. ej., 'Cash Ratio', 'Quick Ratio'),


--- df_prices ---

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   date    851264 non-null  object 
 1   symbol  851264 non-null  object 
 2   open    851264 non-null  float64
 3   close   851264 non-null  float64
 4   low     851264 non-null  float64
 5   high    851264 non-null  float64
 6   volume  851264 non-null  float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB

Valores nulos por columna:
date      0
symbol    0
open      0
close     0
low       0
high      0
volume    0
dtype: int64


--- df_securities ---

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Ticker symbol            505 non-null    object
 1   Security                 505 non-null    object
 2   SEC f

In [87]:
"""
Este bloque de código describe estadísticamente (df.describe(include="all")) cada DataFrame:
1. df_prices: Valores estadísticos de precios y volúmenes.
2. df_securities: Distribución de variables categóricas (sector, dirección, etc.).
3. df_fundamentals: Rango y dispersión de indicadores financieros.

Conclusión final:
- En df_prices, destaca la gran dispersión de precios (1.5 a 1600) y volúmenes (0 a ~8.6e8).
- En df_securities, la mayoría de campos están completos salvo 'Date first added', donde hay ~198 nulos.
- En df_fundamentals, hay múltiples valores atípicos (ROE extremo), columnas con valores negativos muy grandes, y faltantes en columnas clave como 'Cash Ratio' o 'Earnings Per Share'. Deberemos decidir cómo manejar outliers y nulos antes de un análisis o modelado posterior.
"""

dfs = [
    ("df_prices", df_prices),
    ("df_securities", df_securities),
    ("df_fundamentals", df_fundamentals)
]

for name, df in dfs:
    print(f"--- {name}: describe(include='all') ---")
    display(df.describe(include="all"))
    print("\n" + "="*80 + "\n")


--- df_prices: describe(include='all') ---


,date,symbol,open,close,low,high,volume
count,851264,851264,851264.000000,851264.000000,851264.000000,851264.000000,8.512640e+05
unique,1762,501,NaN,NaN,NaN,NaN,NaN
top,2016-07-27,KSU,NaN,NaN,NaN,NaN,NaN
freq,501,1762,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,64.993618,65.011913,64.336541,65.639748,5.415113e+06
std,NaN,NaN,75.203893,75.201216,74.459518,75.906861,1.249468e+07
min,NaN,NaN,1.660000,1.590000,1.500000,1.810000,0.000000e+00
25%,NaN,NaN,31.270000,31.292776,30.940001,31.620001,1.221500e+06
50%,NaN,NaN,48.459999,48.480000,47.970001,48.959999,2.476250e+06
75%,NaN,NaN,75.120003,75.139999,74.400002,75.849998,5.222500e+06




--- df_securities: describe(include='all') ---


,Ticker symbol,Security,SEC filings,GICS Sector,GICS Sub Industry,Address of Headquarters,Date first added,CIK
count,505,505,505,505,505,505,307,5.050000e+02
unique,505,504,1,11,124,264,252,NaN
top,MMM,Under Armour,reports,Consumer Discretionary,Industrial Conglomerates,"New York, New York",1976-06-30,NaN
freq,1,2,505,85,21,47,10,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.074490e+05
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.144375e+05
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.800000e+03
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.631200e+04
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.310010e+05
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.075531e+06




--- df_fundamentals: describe(include='all') ---


,Unnamed: 0,Ticker Symbol,Period Ending,Accounts Payable,Accounts Receivable,Add'l income/expense items,After Tax ROE,Capital Expenditures,Capital Surplus,Cash Ratio,...,Total Current Assets,Total Current Liabilities,Total Equity,Total Liabilities,Total Liabilities & Equity,Total Revenue,Treasury Stock,For Year,Earnings Per Share,Estimated Shares Outstanding
count,1781.000000,1781,1781,1.781000e+03,1.781000e+03,1.781000e+03,1781.000000,1.781000e+03,1.781000e+03,1482.000000,...,1.781000e+03,1.781000e+03,1.781000e+03,1.781000e+03,1.781000e+03,1.781000e+03,1.781000e+03,1608.000000,1562.000000,1.562000e+03
unique,NaN,448,162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,AAL,2014-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,4,317,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,890.000000,NaN,NaN,4.673418e+09,-6.353484e+07,6.908940e+07,43.601348,-1.251925e+09,5.351784e+09,74.457490,...,6.726349e+09,4.699919e+09,1.188846e+10,4.380420e+10,5.569266e+10,2.029154e+10,-3.951522e+09,2013.305348,3.353707,6.024244e+08
std,514.274732,NaN,NaN,1.406047e+10,7.562794e+08,6.848143e+08,233.924028,2.979963e+09,1.133977e+10,102.298374,...,1.344800e+10,9.538832e+09,2.584429e+10,1.817039e+11,2.033612e+11,4.095355e+10,1.407047e+10,19.953135,4.695896,1.142585e+09
min,0.000000,NaN,NaN,0.000000e+00,-6.452000e+09,-6.768000e+09,0.000000,-3.798500e+10,-7.215000e+08,0.000000,...,0.000000e+00,0.000000e+00,-1.324400e+10,2.577000e+06,2.705000e+06,1.514000e+06,-2.297340e+11,1215.000000,-61.200000,-1.513945e+09
25%,445.000000,NaN,NaN,5.160000e+08,-1.040000e+08,-2.638000e+06,10.000000,-1.151200e+09,4.791000e+08,17.000000,...,1.044178e+09,5.640760e+08,2.201492e+09,3.843300e+09,6.552689e+09,3.714000e+09,-3.040895e+09,2013.000000,1.590000,1.493318e+08
50%,890.000000,NaN,NaN,1.334000e+09,-1.830000e+07,2.000000e+06,16.000000,-3.580000e+08,1.997080e+09,41.000000,...,2.747200e+09,1.701500e+09,4.983000e+09,9.141000e+09,1.517000e+10,8.023200e+09,-3.068350e+08,2014.000000,2.810000,2.929409e+08
75%,1335.000000,NaN,NaN,3.246000e+09,7.816000e+06,3.359200e+07,26.000000,-1.291000e+08,5.735000e+09,90.000000,...,6.162000e+09,4.381000e+09,1.080900e+10,2.389700e+10,3.599710e+10,1.748600e+10,0.000000e+00,2015.000000,4.590000,5.492163e+08


In [88]:
#Convertimos Column date a  "datetime64[ns]"
df_prices['date'] = pd.to_datetime(df_prices['date'])
df_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   date    851264 non-null  datetime64[ns]
 1   symbol  851264 non-null  object        
 2   open    851264 non-null  float64       
 3   close   851264 non-null  float64       
 4   low     851264 non-null  float64       
 5   high    851264 non-null  float64       
 6   volume  851264 non-null  float64       
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 45.5+ MB


In [89]:
#No observamos que existan una gran cantidad de Volumen "0" en el dataset de prices. Esto puede ser porque no hay transacciones en ese dia.
#Lo tomomamos como valido
df_prices[df_prices['volume'] == 0]['symbol'].value_counts()


symbol
CHTR    18
VRTX     1
CB       1
SNI      1
Name: count, dtype: int64

In [90]:
#Duplicados: Verificar si existen filas duplicadas de (date, symbol). Si no existen, devolveremos 0 
df_prices.duplicated(subset=['date','symbol']).sum()

0

In [91]:
#Rango temporal del Dataset df_prices : desde 2010-01-04 00:00:00 hasta 2016-12-30 00:00:00
df_prices['date'].min(), df_prices['date'].max()

(Timestamp('2010-01-04 00:00:00'), Timestamp('2016-12-30 00:00:00'))

In [92]:
"""
Eliminamos:
- 'Date first added' y 'SEC filings' de df_securities
- 'Unnamed: 0' y 'For Year' de df_fundamentals
por considerarlas innecesarias para el análisis.
"""

# Eliminamos columnas de df_securities
df_securities.drop(columns=["Date first added", "SEC filings"], inplace=True)

# Eliminamos columnas de df_fundamentals
df_fundamentals.drop(columns=["Unnamed: 0", "For Year"], inplace=True)

# Verificamos las columnas restantes
print("Columnas restantes en df_securities:", df_securities.columns)
print("Columnas restantes en df_fundamentals:", df_fundamentals.columns)


Columnas restantes en df_securities: Index(['Ticker symbol', 'Security', 'GICS Sector', 'GICS Sub Industry',
       'Address of Headquarters', 'CIK'],
      dtype='object')
Columnas restantes en df_fundamentals: Index(['Ticker Symbol', 'Period Ending', 'Accounts Payable',
       'Accounts Receivable', 'Add'l income/expense items', 'After Tax ROE',
       'Capital Expenditures', 'Capital Surplus', 'Cash Ratio',
       'Cash and Cash Equivalents', 'Changes in Inventories', 'Common Stocks',
       'Cost of Revenue', 'Current Ratio', 'Deferred Asset Charges',
       'Deferred Liability Charges', 'Depreciation',
       'Earnings Before Interest and Tax', 'Earnings Before Tax',
       'Effect of Exchange Rate',
       'Equity Earnings/Loss Unconsolidated Subsidiary', 'Fixed Assets',
       'Goodwill', 'Gross Margin', 'Gross Profit', 'Income Tax',
       'Intangible Assets', 'Interest Expense', 'Inventory', 'Investments',
       'Liabilities', 'Long-Term Debt', 'Long-Term Investments',
      

In [93]:
# Creamos una columna booleana 'has_eps' indicando si existe un valor válido en 'Earnings Per Share'
df_fundamentals["has_eps"] = df_fundamentals["Earnings Per Share"].notna()

# Mostramos las primeras filas para comprobar la creación y valores de la nueva columna
print(df_fundamentals[["Earnings Per Share", "has_eps"]].head(10))

# Resumen de las columnas y su tipo
df_fundamentals.info()

# Mostramos las columnas restantes en forma de lista para mayor claridad
print("\nColumnas actuales en df_fundamentals:")
print(df_fundamentals.columns.to_list())

# Opcional: muestra los primeros registros para verificar
print("\nPrimeras filas de df_fundamentals tras los cambios:")
display(df_fundamentals.head())



   Earnings Per Share  has_eps
0               -5.60     True
1              -11.25     True
2                4.02     True
3               11.39     True
4                5.29     True
5                5.36     True
6                6.75     True
7                6.45     True
8               40.03     True
9                6.49     True
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1781 entries, 0 to 1780
Data columns (total 78 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   Ticker Symbol                                        1781 non-null   object 
 1   Period Ending                                        1781 non-null   object 
 2   Accounts Payable                                     1781 non-null   float64
 3   Accounts Receivable                                  1781 non-null   float64
 4   Add'l income/expense items                           17

,Ticker Symbol,Period Ending,Accounts Payable,Accounts Receivable,Add'l income/expense items,After Tax ROE,Capital Expenditures,Capital Surplus,Cash Ratio,Cash and Cash Equivalents,...,Total Current Assets,Total Current Liabilities,Total Equity,Total Liabilities,Total Liabilities & Equity,Total Revenue,Treasury Stock,Earnings Per Share,Estimated Shares Outstanding,has_eps
0,AAL,2012-12-31,3.068000e+09,-222000000.0,-1.961000e+09,23.0,-1.888000e+09,4.695000e+09,53.0,1.330000e+09,...,7.072000e+09,9.011000e+09,-7.987000e+09,2.489100e+10,1.690400e+10,2.485500e+10,-367000000.0,-5.60,3.350000e+08,True
1,AAL,2013-12-31,4.975000e+09,-93000000.0,-2.723000e+09,67.0,-3.114000e+09,1.059200e+10,75.0,2.175000e+09,...,1.432300e+10,1.380600e+10,-2.731000e+09,4.500900e+10,4.227800e+10,2.674300e+10,0.0,-11.25,1.630222e+08,True
2,AAL,2014-12-31,4.668000e+09,-160000000.0,-1.500000e+08,143.0,-5.311000e+09,1.513500e+10,60.0,1.768000e+09,...,1.175000e+10,1.340400e+10,2.021000e+09,4.120400e+10,4.322500e+10,4.265000e+10,0.0,4.02,7.169154e+08,True
3,AAL,2015-12-31,5.102000e+09,352000000.0,-7.080000e+08,135.0,-6.151000e+09,1.159100e+10,51.0,1.085000e+09,...,9.985000e+09,1.360500e+10,5.635000e+09,4.278000e+10,4.841500e+10,4.099000e+10,0.0,11.39,6.681299e+08,True
4,AAP,2012-12-29,2.409453e+09,-89482000.0,6.000000e+05,32.0,-2.711820e+08,5.202150e+08,23.0,5.981110e+08,...,3.184200e+09,2.559638e+09,1.210694e+09,3.403120e+09,4.613814e+09,6.205003e+09,-27095000.0,5.29,7.328355e+07,True


In [94]:
# Renombramos columnas clave en cada DF (si no lo habías hecho antes)

df_prices.rename(columns={"symbol": "ticker"}, inplace=True)
df_securities.rename(columns={"Ticker symbol": "ticker"}, inplace=True)
df_fundamentals.rename(columns={"Ticker Symbol": "ticker"}, inplace=True)


In [95]:
# Paso 1: Asegurarse de que la columna 'date' en df_prices es datetime y extraer el año
df_prices['date'] = pd.to_datetime(df_prices['date'])
df_prices['fiscal_year'] = df_prices['date'].dt.year

# Paso 2: Convertir "Period Ending" en df_fundamentals a datetime y extraer el año
df_fundamentals['Period Ending'] = pd.to_datetime(df_fundamentals['Period Ending'])
df_fundamentals['fiscal_year'] = df_fundamentals['Period Ending'].dt.year

# Opcional: Revisar las nuevas columnas para confirmar la extracción
print("Ejemplo en df_prices:")
print(df_prices[['date', 'fiscal_year']].head())
print("\nEjemplo en df_fundamentals:")
print(df_fundamentals[['Period Ending', 'fiscal_year']].head())

# Paso 3: Realizar el join de los DataFrames
# Primero, unimos df_prices con df_securities (ambos ya deben tener la columna 'ticker' homogenizada)
df_merged = pd.merge(df_prices, df_securities, on="ticker", how="inner")
print("\ndf_merged shape:", df_merged.shape)

# Luego, unimos el resultado con df_fundamentals utilizando tanto 'ticker' como 'fiscal_year'
df_final = pd.merge(df_merged, df_fundamentals, on=["ticker", "fiscal_year"], how="inner")

print("\ndf_final shape:", df_final.shape)
display(df_final.head())


Ejemplo en df_prices:
        date  fiscal_year
0 2016-01-05         2016
1 2016-01-06         2016
2 2016-01-07         2016
3 2016-01-08         2016
4 2016-01-11         2016

Ejemplo en df_fundamentals:
  Period Ending  fiscal_year
0    2012-12-31         2012
1    2013-12-31         2013
2    2014-12-31         2014
3    2015-12-31         2015
4    2012-12-29         2012

df_merged shape: (851264, 13)

df_final shape: (439158, 90)


,date,ticker,open,close,low,high,volume,fiscal_year,Security,GICS Sector,...,Total Current Assets,Total Current Liabilities,Total Equity,Total Liabilities,Total Liabilities & Equity,Total Revenue,Treasury Stock,Earnings Per Share,Estimated Shares Outstanding,has_eps
0,2012-01-03,AAL,5.200000,5.120000,5.070000,5.220000,6105900.0,2012,American Airlines Group,Industrials,...,7.072000e+09,9.011000e+09,-7.987000e+09,2.489100e+10,1.690400e+10,2.485500e+10,-3.670000e+08,-5.60,3.350000e+08,True
1,2012-01-03,AAP,71.139999,69.099998,69.059998,71.599998,786600.0,2012,Advance Auto Parts,Consumer Discretionary,...,3.184200e+09,2.559638e+09,1.210694e+09,3.403120e+09,4.613814e+09,6.205003e+09,-2.709500e+07,5.29,7.328355e+07,True
2,2012-01-03,ABT,27.147059,27.214232,26.897565,27.271808,17693200.0,2012,Abbott Laboratories,Health Care,...,3.132300e+10,1.328000e+10,2.672100e+10,4.051400e+10,6.723500e+10,1.905000e+10,-5.591000e+09,3.76,1.585904e+09,True
3,2012-01-03,ADM,29.190001,28.889999,28.879999,29.320000,4012500.0,2012,Archer-Daniels-Midland Co,Consumer Staples,...,2.976200e+10,1.699300e+10,1.892000e+10,2.600500e+10,4.492500e+10,9.055900e+10,0.000000e+00,NaN,NaN,False
4,2012-01-03,ADS,102.989998,102.760002,102.040001,105.250000,1661400.0,2012,Alliance Data Systems,Information Technology,...,9.132143e+09,5.032777e+09,5.284870e+08,1.147165e+10,1.200014e+10,3.641390e+09,-2.458092e+09,8.44,5.003033e+07,True


In [96]:
# Filtramos df_final para obtener solo las filas correspondientes a Apple (ticker "AAPL")
# Se analizo APPL Para ver en detalle la apertura de este nuevo DF
df_apple = df_final[df_final["ticker"] == "AAPL"]

# Exportamos el extracto a un archivo CSV sin el índice
df_apple.to_csv("df_apple_final_extract.csv", index=False)


In [97]:
"""
Este bloque de código realiza lo siguiente:
1. Convierte la columna 'date' a Period (mes) para agrupar los datos diarios.
2. Calcula el promedio mensual de 'open', 'close', 'low', 'high'
   y suma el 'volume' para cada mes.
3. Extrae el año (fiscal_year) a partir del periodo mensual, para luego
   poder unir (join) con los datos fundamentales que tienen esa misma referencia de año.
"""

# Asegurarse de que 'date' está en formato datetime
df_prices['date'] = pd.to_datetime(df_prices['date'])

# Crear una columna 'month' como Periodo (Mes)
df_prices['month'] = df_prices['date'].dt.to_period('M')

# Agrupar por 'month' y calcular el promedio mensual de precios y la suma del volumen
monthly_avg = df_prices.groupby('month').agg({
    'open': 'mean',
    'close': 'mean',
    'low': 'mean',
    'high': 'mean',
    'volume': 'sum'
}).reset_index()

# Extraer el año fiscal a partir del periodo mensual
monthly_avg['fiscal_year'] = monthly_avg['month'].dt.year

# Mostrar el resultado
print("Promedio mensual de precios:")
print(monthly_avg.head())


Promedio mensual de precios:
     month       open      close        low       high        volume  \
0  2010-01  38.981831  38.893568  38.433616  39.405804  7.041815e+10   
1  2010-02  37.760727  37.845064  37.321445  38.200534  6.597997e+10   
2  2010-03  40.262621  40.340871  39.925267  40.678240  7.330653e+10   
3  2010-04  42.186109  42.239189  41.720681  42.674732  7.527794e+10   
4  2010-05  40.225591  40.140559  39.394016  40.852070  9.128969e+10   

   fiscal_year  
0         2010  
1         2010  
2         2010  
3         2010  
4         2010  


In [98]:
"""
1. Convertimos 'date' a datetime y creamos la columna 'month' (periodo mensual).
2. Agrupamos por 'ticker' y 'month' para calcular el promedio de precios y sumar el volumen.
3. Extraemos el año fiscal a partir del mes para alinearlo con los balances.
4. Realizamos el merge: primero unimos con df_securities (a nivel de ticker) y luego con df_fundamentals (por ticker y fiscal_year).
"""

# Paso 1: Asegurarse que 'date' está en formato datetime y crear la columna 'month'
df_prices['date'] = pd.to_datetime(df_prices['date'])
df_prices['month'] = df_prices['date'].dt.to_period('M')

# Paso 2: Agrupar por 'ticker' y 'month'
monthly_avg = df_prices.groupby(['ticker', 'month']).agg({
    'open': 'mean',
    'close': 'mean',
    'low': 'mean',
    'high': 'mean',
    'volume': 'sum'
}).reset_index()

# Paso 3: Extraer el año fiscal
monthly_avg['fiscal_year'] = monthly_avg['month'].dt.year

print("Extracto del promedio mensual por ticker:")
display(monthly_avg.head())

# Paso 4: Unir con df_securities (por 'ticker')
df_monthly = pd.merge(monthly_avg, df_securities, on="ticker", how="inner")
print("df_monthly shape:", df_monthly.shape)
print("Columnas en df_monthly:", df_monthly.columns)

# Paso 5: Unir con df_fundamentals (por 'ticker' y 'fiscal_year')
df_final = pd.merge(df_monthly, df_fundamentals, on=["ticker", "fiscal_year"], how="inner")
print("df_final shape:", df_final.shape)
print("Columnas en df_final:", df_final.columns)
display(df_final.head())


Extracto del promedio mensual por ticker:


,ticker,month,open,close,low,high,volume,fiscal_year
0,A,2010-01,21.718244,21.610195,21.402003,21.856035,79960400.0,2010
1,A,2010-02,21.476169,21.551465,21.227317,21.701679,108262400.0,2010
2,A,2010-03,23.864527,23.954407,23.732350,24.073521,120152900.0,2010
3,A,2010-04,25.263301,25.331766,25.042237,25.525921,106279800.0,2010
4,A,2010-05,23.817239,23.756080,23.310086,24.183476,124701400.0,2010


df_monthly shape: (40587, 13)
Columnas en df_monthly: Index(['ticker', 'month', 'open', 'close', 'low', 'high', 'volume',
       'fiscal_year', 'Security', 'GICS Sector', 'GICS Sub Industry',
       'Address of Headquarters', 'CIK'],
      dtype='object')
df_final shape: (20937, 90)
Columnas en df_final: Index(['ticker', 'month', 'open', 'close', 'low', 'high', 'volume',
       'fiscal_year', 'Security', 'GICS Sector', 'GICS Sub Industry',
       'Address of Headquarters', 'CIK', 'Period Ending', 'Accounts Payable',
       'Accounts Receivable', 'Add'l income/expense items', 'After Tax ROE',
       'Capital Expenditures', 'Capital Surplus', 'Cash Ratio',
       'Cash and Cash Equivalents', 'Changes in Inventories', 'Common Stocks',
       'Cost of Revenue', 'Current Ratio', 'Deferred Asset Charges',
       'Deferred Liability Charges', 'Depreciation',
       'Earnings Before Interest and Tax', 'Earnings Before Tax',
       'Effect of Exchange Rate',
       'Equity Earnings/Loss Unconso

,ticker,month,open,close,low,high,volume,fiscal_year,Security,GICS Sector,...,Total Current Assets,Total Current Liabilities,Total Equity,Total Liabilities,Total Liabilities & Equity,Total Revenue,Treasury Stock,Earnings Per Share,Estimated Shares Outstanding,has_eps
0,AAL,2012-01,6.308500,6.416000,6.194500,6.565000,185238100.0,2012,American Airlines Group,Industrials,...,7.072000e+09,9.011000e+09,-7.987000e+09,2.489100e+10,1.690400e+10,2.485500e+10,-367000000.0,-5.6,335000000.0,True
1,AAL,2012-02,8.550000,8.484500,8.285000,8.750000,180522000.0,2012,American Airlines Group,Industrials,...,7.072000e+09,9.011000e+09,-7.987000e+09,2.489100e+10,1.690400e+10,2.485500e+10,-367000000.0,-5.6,335000000.0,True
2,AAL,2012-03,7.329091,7.337273,7.163182,7.528182,157353600.0,2012,American Airlines Group,Industrials,...,7.072000e+09,9.011000e+09,-7.987000e+09,2.489100e+10,1.690400e+10,2.485500e+10,-367000000.0,-5.6,335000000.0,True
3,AAL,2012-04,8.446000,8.554500,8.295000,8.733000,182375100.0,2012,American Airlines Group,Industrials,...,7.072000e+09,9.011000e+09,-7.987000e+09,2.489100e+10,1.690400e+10,2.485500e+10,-367000000.0,-5.6,335000000.0,True
4,AAL,2012-05,11.067273,11.184545,10.857727,11.402273,187875000.0,2012,American Airlines Group,Industrials,...,7.072000e+09,9.011000e+09,-7.987000e+09,2.489100e+10,1.690400e+10,2.485500e+10,-367000000.0,-5.6,335000000.0,True


In [101]:
# Filtramos el DataFrame final para obtener solo las filas correspondientes a Apple (ticker "AAPL")
df_apple = df_final[df_final["ticker"] == "AAPL"]

# Mostramos las primeras filas para verificar el extracto
display(df_apple.head())

# Imprimimos la cantidad de registros para Apple
print("Cantidad de registros para AAPL:", df_apple.shape[0])

# Exportamos el extracto a un archivo CSV sin el índice
df_apple.to_csv("df_apple_monthly_extract.csv", index=False)
print("Archivo 'df_apple_monthly_extract.csv' generado correctamente.")


,ticker,month,open,close,low,high,volume,fiscal_year,Security,GICS Sector,...,Total Current Assets,Total Current Liabilities,Total Equity,Total Liabilities,Total Liabilities & Equity,Total Revenue,Treasury Stock,Earnings Per Share,Estimated Shares Outstanding,has_eps
96,AAPL,2013-01,71.440476,71.117483,70.424286,72.058231,3.280856e+09,2013,Apple Inc.,Information Technology,...,7.328600e+10,4.365800e+10,1.235490e+11,8.345100e+10,2.070000e+11,1.709100e+11,0.0,40.03,9.252311e+08,True
97,AAPL,2013-02,65.500527,65.258422,64.586617,66.175865,2.336008e+09,2013,Apple Inc.,Information Technology,...,7.328600e+10,4.365800e+10,1.235490e+11,8.345100e+10,2.070000e+11,1.709100e+11,0.0,40.03,9.252311e+08,True
98,AAPL,2013-03,63.144501,63.120143,62.476858,63.829071,2.294219e+09,2013,Apple Inc.,Information Technology,...,7.328600e+10,4.365800e+10,1.235490e+11,8.345100e+10,2.070000e+11,1.709100e+11,0.0,40.03,9.252311e+08,True
99,AAPL,2013-04,59.966688,59.966428,59.246039,60.793312,2.740872e+09,2013,Apple Inc.,Information Technology,...,7.328600e+10,4.365800e+10,1.235490e+11,8.345100e+10,2.070000e+11,1.709100e+11,0.0,40.03,9.252311e+08,True
100,AAPL,2013-05,63.771558,63.778961,63.070779,64.522598,2.361883e+09,2013,Apple Inc.,Information Technology,...,7.328600e+10,4.365800e+10,1.235490e+11,8.345100e+10,2.070000e+11,1.709100e+11,0.0,40.03,9.252311e+08,True


Cantidad de registros para AAPL: 48
Archivo 'df_apple_monthly_extract.csv' generado correctamente.


In [107]:
import sys
!{sys.executable} -m pip install ydata-profiling


^C


In [103]:
# Importamos la librería
from ydata_profiling import ProfileReport

# Generamos el perfil (informe) para tu DataFrame final
profile = ProfileReport(
    df_final, 
    title="Análisis Exploratorio - df_final",
    explorative=True  # habilita opciones interactivas y un análisis más detallado
)

# Mostrar el reporte directamente en Jupyter/Colab
profile


ModuleNotFoundError: No module named 'ydata_profiling'